In [2]:
# Import required packages
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import numpy as np
from tensorflow import keras
import tensorflow.keras as K
import cv2
import tensorflow_hub as hub
import os
import PIL
from datetime import datetime
import collections
import timeit

## 1. Load the datasets

For the project, we provide a training set with 50000 images in the directory `../data/images/` with:
- noisy labels for all images provided in `../data/noisy_label.csv`;
- clean labels for the first 10000 images provided in `../data/clean_labels.csv`.

In [3]:
# [DO NOT MODIFY THIS CELL]

# load the images
n_img = 50000
n_noisy = 40000
n_clean_noisy = n_img - n_noisy
imgs = np.empty((n_img,32,32,3))
for i in range(n_img):
    img_fn = f'../data/images/{i+1:05d}.png'
    imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)

# load the labels
clean_labels = np.genfromtxt('../data/clean_labels.csv', delimiter=',', dtype="int8")
noisy_labels = np.genfromtxt('../data/noisy_labels.csv', delimiter=',', dtype="int8")

For illustration, we present a small subset (of size 8) of the images with their clean and noisy labels in `clean_noisy_trainset`. You are encouraged to explore more characteristics of the label noises on the whole dataset. 

In [ ]:
# [DO NOT MODIFY THIS CELL]

fig = plt.figure()

ax1 = fig.add_subplot(2,4,1)
ax1.imshow(imgs[0]/255)
ax2 = fig.add_subplot(2,4,2)
ax2.imshow(imgs[1]/255)
ax3 = fig.add_subplot(2,4,3)
ax3.imshow(imgs[2]/255)
ax4 = fig.add_subplot(2,4,4)
ax4.imshow(imgs[3]/255)
ax1 = fig.add_subplot(2,4,5)
ax1.imshow(imgs[4]/255)
ax2 = fig.add_subplot(2,4,6)
ax2.imshow(imgs[5]/255)
ax3 = fig.add_subplot(2,4,7)
ax3.imshow(imgs[6]/255)
ax4 = fig.add_subplot(2,4,8)
ax4.imshow(imgs[7]/255)

# The class-label correspondence
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# print clean labels
print('Clean labels:')
print(' '.join('%5s' % classes[clean_labels[j]] for j in range(10)))
# print noisy labels
print('Noisy labels:')
print(' '.join('%5s' % classes[noisy_labels[j]] for j in range(10)))


## 2. The predictive model

We consider a baseline model directly on the noisy dataset without any label corrections. RGB histogram features are extracted to fit a logistic regression model.

### 2.1. Baseline Model

In [ ]:
# [DO NOT MODIFY THIS CELL]
# RGB histogram dataset construction
no_bins = 6
bins = np.linspace(0,255,no_bins) # the range of the rgb histogram
target_vec = np.empty(n_img)
feature_mtx = np.empty((n_img,3*(len(bins)-1)))
i = 0
for i in range(n_img):
    # The target vector consists of noisy labels
    target_vec[i] = noisy_labels[i]
    
    # Use the numbers of pixels in each bin for all three channels as the features
    feature1 = np.histogram(imgs[i][:,:,0],bins=bins)[0] 
    feature2 = np.histogram(imgs[i][:,:,1],bins=bins)[0]
    feature3 = np.histogram(imgs[i][:,:,2],bins=bins)[0]
    
    # Concatenate three features
    feature_mtx[i,] = np.concatenate((feature1, feature2, feature3), axis=None)
    i += 1

In [ ]:
# [DO NOT MODIFY THIS CELL]
# Train a logistic regression model 
clf = LogisticRegression(random_state=0).fit(feature_mtx, target_vec)

For the convenience of evaluation, we write the following function `predictive_model` that does the label prediction. **For your predictive model, feel free to modify the function, but make sure the function takes an RGB image of numpy.array format with dimension $32\times32\times3$  as input, and returns one single label as output.**

In [ ]:
# [DO NOT MODIFY THIS CELL]
def baseline_model(image):
    '''
    This is the baseline predictive model that takes in the image and returns a label prediction
    '''
    feature1 = np.histogram(image[:,:,0],bins=bins)[0]
    feature2 = np.histogram(image[:,:,1],bins=bins)[0]
    feature3 = np.histogram(image[:,:,2],bins=bins)[0]
    feature = np.concatenate((feature1, feature2, feature3), axis=None).reshape(1,-1)
    return clf.predict(feature)

### 2.2 Model I (Treating noisy labels as clean)

##### Train-test data split

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(imgs[:10000], clean_labels, test_size=0.2, random_state=42)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

X_train_40k = imgs[10000:] / 255.0
y_train_40k = noisy_labels[10000:]

X_valid_10k = imgs[:10000] / 255.0
y_valid_10k = clean_labels

#X_valid, X_test, y_valid, y_test = train_test_split(imgs[:10000], clean_labels, test_size=0.5, random_state=42)
#X_valid = np.array(X_valid) / 255.0

In [ ]:
X_train_40k

In [ ]:
#X_train_all = np.concatenate((X_train, X_train_noisy))
#y_train_all = np.concatenate((y_train, y_train_noisy))
#X_train_all = np.array(X_train_all) / 255.0
#X_valid = np.array(X_valid) / 255.0

#### 2.2.1 CNN without data augmentation

In [ ]:
# https://www.tensorflow.org/tutorials/images/cnn
cnn = Sequential()
cnn.add(layers.Conv2D(32, (3,3), padding="same", activation="relu", input_shape=(32, 32, 3)))
cnn.add(layers.MaxPooling2D(2, 2))
cnn.add(layers.Conv2D(64, (3,3), padding="same", activation="relu"))
cnn.add(layers.MaxPooling2D(2, 2))
cnn.add(layers.Conv2D(64, (3,3), padding="same", activation="relu"))

# add dense layers on top
cnn.add(layers.Flatten())
cnn.add(layers.Dense(64, activation='relu'))
cnn.add(layers.Dense(10))

# compile the model
cnn.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
# set parameters for autosaving the model with the best validation accuracy
# reduce learning rate when val_accuracy has stopped improving
lr_reduce = K.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                          factor=0.6,
                                          patience=2,
                                          verbose=1,
                                          mode='max',
                                          min_lr=1e-7)
# stop training when val_accuracy has stopped improving
early_stop = K.callbacks.EarlyStopping(monitor='val_accuracy',
                                       patience=3,
                                       verbose=1,
                                       mode='max')
# callback to save the Keras model and (best) weights obtained on an epoch basis. here, the trained (compiled) model is saved in the current working directory as ''
filepath_cnn = '../output/saved_model/cnn'
checkpoint = K.callbacks.ModelCheckpoint(filepath=filepath_cnn, # remember to change the path
                                         monitor='val_accuracy',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         save_freq='epoch')

In [ ]:
#train the model and autosave the model with the best validation accuracy
cnn_history = cnn.fit(X_train_40k,
                      y_train_40k, 
                      epochs = 20, 
                      validation_data=(X_valid_10k, y_valid_10k),
                      shuffle=True,
                      callbacks=[lr_reduce, early_stop, checkpoint],
                      verbose=1)

In [ ]:
# Save the fit history
import pickle
with open('../output/evaluation_model/cnn_hisotry', 'wb') as fp:
    pickle.dump(cnn_history, fp)
    
# Load the history fit back from the pickle file.
# import pickle
# cnn_history = pickle.load(open("../output/evaluation_model/cnn_hisotry", "rb"))

In [ ]:
# load the model
filepath_cnn = '../output/saved_model/cnn'
cnn = K.models.load_model(filepath=filepath_cnn)

Evaluate the model

In [ ]:
#1. time
start = timeit.default_timer()

y_pred = cnn.predict(X_valid_10k)

stop = timeit.default_timer()
print(f'-----The Model takes {stop - start:.2f} seconds to run 10k predictions-----')

In [ ]:
# plot the training history
# 2. Validation accuracy
# plt.plot(cnn_history.history['accuracy'], label = 'accuracy')
plt.plot(cnn_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.savefig('../figs/cnn_accuracy')
plt.show()

In [ ]:
# 3. Validation loss
# plt.plot(cnn_history.history['loss'], label='Training Loss')
plt.plot(cnn_history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.savefig('../figs/cnn_loss')
plt.show()

#Load and evaluate the best model version
# model = load_model(filepath)
# yhat = model.predict(X_test)
# print('Model MSE on test data = ', mse(y_test, yhat).numpy())

#### 2.2.2 CNN with data augmentation

In [ ]:
# [BUILD A MORE SOPHISTICATED PREDICTIVE MODEL]

# data augomentation
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(32, 
                                                              32,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)


cnn_aug = Sequential([
  data_augmentation,
  layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10)
    ])

# compile the model
cnn_aug.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [ ]:
# set parameters for autosaving the model with the best validation accuracy
filepath_cnn_aug = '../output/saved_model/cnn_aug'
# reduce learning rate when val_accuracy has stopped improving
lr_reduce = K.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                          factor=0.6,
                                          patience=2,
                                          verbose=1,
                                          mode='max',
                                          min_lr=1e-7)
# stop training when val_accuracy has stopped improving
early_stop = K.callbacks.EarlyStopping(monitor='val_accuracy',
                                       patience=3,
                                       verbose=1,
                                       mode='max')
# callback to save the Keras model and (best) weights obtained on an epoch basis. here, the trained (compiled) model is saved in the current working directory as ''
checkpoint = K.callbacks.ModelCheckpoint(filepath=filepath_cnn_aug, # remember to change the path
                                         monitor='val_accuracy',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         save_freq='epoch')

In [ ]:
#train the model and autosave the model with the best validation accuracy
cnn_aug_history = cnn_aug.fit(X_train_40k,
                      y_train_40k, 
                      epochs = 20, 
                      validation_data=(X_valid_10k, y_valid_10k),
                      shuffle=True,
                      callbacks=[lr_reduce, early_stop, checkpoint],
                      verbose=1)

In [ ]:
# Save the fit history
import pickle
with open('../output/evaluation_model/cnn_aug_hisotry', 'wb') as fp:
    pickle.dump(cnn_aug_history, fp)
    
# Load the history fit back from the pickle file.
# import pickle
# cnn_history = pickle.load(open("../output/evaluation_model/cnn_aug_hisotry", "rb"))

In [ ]:
# load the model
filepath_cnn_aug = '../output/saved_model/cnn_aug'
cnn_aug = K.models.load_model(filepath=filepath_cnn_aug)

Evaluate the model

In [ ]:
#1. time
start = timeit.default_timer()

y_pred = cnn_aug.predict(X_valid_10k)

stop = timeit.default_timer()
print(f'-----The Model takes {stop - start:.2f} seconds to run 10k predictions-----')

In [ ]:
# plot the training history
# 2. Validation accuracy
# plt.plot(cnn_aug_history.history['accuracy'], label = 'accuracy')
plt.plot(cnn_aug_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.savefig('../figs/cnn_aug_accuracy')
plt.show()

In [ ]:
# 3. Validation loss
# plt.plot(cnn_aug_history.history['loss'], label='Training Loss')
plt.plot(cnn_aug_history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.savefig('../figs/cnn_aug_loss')
plt.show()

#Load and evaluate the best model version
# model = load_model(filepath)
# yhat = model.predict(X_test)
# print('Model MSE on test data = ', mse(y_test, yhat).numpy())

After comparing the performance of CNN without data augmentation and CNN without data augmentation, we choose CNN with data augmentation as our Model I

In [ ]:
def model_I(image):
#      '''
#     This function should takes in the image of dimension 32*32*3 as input and returns a label prediction
#     '''
    X = image/255.0
    # predict
    y_pred = cnn_aug.predict(X)
    y_pred_labels = [np.argmax(x) for x in y_pred]
    return y_pred_labels

### 2.3. Model II (After cleaning the labels)

#### Correct the noisy labels
We put the code in the '../doc/label_correction_model.ipynb' to keep this notebook tidy.

After the model is created to clean the labels, cleaned labels are saved in output, and they are used in the below model

#### Load the data (cleaned labels)

In [ ]:
cleaned_labels = np.genfromtxt('../output/cleaned_labels.csv', delimiter=',', dtype="int8")
y_train_cleaned = cleaned_labels[10000:]

In [ ]:
# [BUILD A MORE SOPHISTICATED PREDICTIVE MODEL]

# data augomentation
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(32, 
                                                              32,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)


model_2 = Sequential([
  data_augmentation,
  layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10)
    ])

# compile the model
model_2.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [ ]:
# set parameters for autosaving the model with the best validation accuracy
filepath_model_2 = '../output/saved_model/model_2'
# reduce learning rate when val_accuracy has stopped improving
lr_reduce = K.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                          factor=0.6,
                                          patience=2,
                                          verbose=1,
                                          mode='max',
                                          min_lr=1e-7)
# stop training when val_accuracy has stopped improving
early_stop = K.callbacks.EarlyStopping(monitor='val_accuracy',
                                       patience=3,
                                       verbose=1,
                                       mode='max')
# callback to save the Keras model and (best) weights obtained on an epoch basis. here, the trained (compiled) model is saved in the current working directory as ''
checkpoint = K.callbacks.ModelCheckpoint(filepath=filepath_model_2, # remember to change the path
                                         monitor='val_accuracy',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         save_freq='epoch')

In [ ]:
#train the model and autosave the model with the best validation accuracy
model_2_history = model_2.fit(X_train_40k,
                      y_train_cleaned, # use the corrected labels to train the data
                      epochs = 20, 
                      validation_data=(X_valid_10k, y_valid_10k),
                      shuffle=True,
                      callbacks=[lr_reduce, early_stop, checkpoint],
                      verbose=1)

In [ ]:
# Save the fit history
import pickle
with open('../output/evaluation_model/model_2_hisotry', 'wb') as fp:
    pickle.dump(model_2_history, fp)
    
# Load the history fit back from the pickle file.
# import pickle
# cnn_history = pickle.load(open("../output/evaluation_model/cnn_aug_hisotry", "rb"))

In [ ]:
# load the model
filepath_model_2 = '../output/saved_model/model_2'
cnn_aug = K.models.load_model(filepath=filepath_model_2)

In [ ]:
#1. time
start = timeit.default_timer()

y_pred = model_2.predict(X_valid_10k)

stop = timeit.default_timer()
print(f'-----The Model takes {stop - start:.2f} seconds to run 10k predictions-----')

In [ ]:
# plot the training history
# 2. Validation accuracy
# plt.plot(cnn_aug_history.history['accuracy'], label = 'accuracy')
plt.plot(model_2_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.savefig('../figs/model_2_accuracy')
plt.show()

In [ ]:
# 3. Validation loss
# plt.plot(cnn_aug_history.history['loss'], label='Training Loss')
plt.plot(model_2_history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.savefig('../figs/model_2_loss')
plt.show()

#Load and evaluate the best model version
# model = load_model(filepath)
# yhat = model.predict(X_test)
# print('Model MSE on test data = ', mse(y_test, yhat).numpy())

In [ ]:
def model_II(image):
#      '''
#     This function should takes in the image of dimension 32*32*3 as input and returns a label prediction
#     '''
    X = image/255.0
    # predict
    y_pred = model_2.predict(X)
    y_pred_labels = [np.argmax(x) for x in y_pred]
    return y_pred_labels

## 3. Evaluation

For assessment, we will evaluate your final model on a hidden test dataset with clean labels by the `evaluation` function defined as follows. Although you will not have the access to the test set, the function would be useful for the model developments. For example, you can split the small training set, using one portion for weakly supervised learning and the other for validation purpose. 

In [ ]:
# load the model 1
filepath_cnn_aug = '../output/saved_model/cnn_aug'
cnn_aug = K.models.load_model(filepath=filepath_cnn_aug)

In [ ]:
# load the model 2
filepath_model_2 = '../output/saved_model/model_2'
model_2 = K.models.load_model(filepath=filepath_model_2)

In [ ]:
def model_I(image):
#      '''
#     This function should takes in the image of dimension 32*32*3 as input and returns a label prediction
#     '''
    X = image/255.0
    # predict
    y_pred = cnn_aug.predict(X)
    y_pred_labels = [np.argmax(x) for x in y_pred]
    return y_pred_labels

In [ ]:
def model_II(image):
#      '''
#     This function should takes in the image of dimension 32*32*3 as input and returns a label prediction
#     '''
    X = image/255.0
    # predict
    y_pred = model_2.predict(X)
    y_pred_labels = [np.argmax(x) for x in y_pred]
    return y_pred_labels

In [ ]:
# [DO NOT MODIFY THIS CELL]
def evaluation(model, test_labels, test_imgs):
    y_true = test_labels.tolist()
    y_pred = model(test_imgs)
    print(classification_report(y_true, y_pred))

In [ ]:
# For final test
# [DO NOT MODIFY THIS CELL]
# This is the code for evaluating the prediction performance on a testset
# You will get an error if running this cell, as you do not have the testset
# Nonetheless, you can create your own validation set to run the evlauation
# n_test = 10000
# test_labels = np.genfromtxt('../data/test_labels.csv', delimiter=',', dtype="int8")
# test_imgs = np.empty((n_test,32,32,3))
# for i in range(n_test):
#     img_fn = f'../data/test_images/test{i+1:05d}.png'
#     test_imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)
# evaluation(baseline_model, test_labels, test_imgs)

In [ ]:
# This is just for our trial, remember to delete this cell
n_test = 10000
test_labels = np.genfromtxt('../data/clean_labels.csv', delimiter=',', dtype="int8") 
test_imgs = np.empty((n_test,32,32,3))
for i in range(n_test):
    img_fn = f'../data/images/{i+1:05d}.png'
    test_imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)

In [ ]:
# test for CNN with augmentation (Model_I)
start = timeit.default_timer()

evaluation(model_I, test_labels, test_imgs)

stop = timeit.default_timer()
print(f'-----The Model I takes {stop - start:.2f} seconds-----')

In [ ]:
# test for CNN with augmentation (Model_I)
start = timeit.default_timer()

evaluation(model_II, test_labels, test_imgs)

stop = timeit.default_timer()
print(f'-----The Model II takes {stop - start:.2f} seconds-----')

The overall accuracy is $0.24$, which is better than random guess (which should have a accuracy around $0.10$). For the project, you should try to improve the performance by the following strategies:

- Consider a better choice of model architectures, hyperparameters, or training scheme for the predictive model;
- Use both `clean_noisy_trainset` and `noisy_trainset` for model training via **weakly supervised learning** methods. One possible solution is to train a "label-correction" model using the former, correct the labels in the latter, and train the final predictive model using the corrected dataset.
- Apply techniques such as $k$-fold cross validation to avoid overfitting;
- Any other reasonable strategies.